In [6]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from  matplotlib.colors import LinearSegmentedColormap
from tqdm import trange
import pandas as pd
# pd.set_option('expand_frame_repr', False)
from matplotlib.ticker import FormatStrFormatter  
# sns.set()
sns.set(font_scale=2)
import sys
import numpy as np
import os
import re
from scipy.stats import pearsonr

root_path = '/data/'

In [7]:
# 构建 dataset 和 domain 的映射字典
dataset_domain_mapping = {
    # Nature
    "jena_weather/10T": "Nature",
    "jena_weather/H": "Nature",
    "jena_weather/D": "Nature",
    "saugeenday/D": "Nature",
    "saugeenday/M": "Nature",
    "saugeenday/W": "Nature",
    "temperature_rain_with_missing": "Nature",
    
    # Web/CloudOps
    "bitbrains_fast_storage/5T": "Web/CloudOps",
    "bitbrains_fast_storage/H": "Web/CloudOps",
    "bitbrains_rnd/5T": "Web/CloudOps",
    "bitbrains_rnd/H": "Web/CloudOps",
    "bizitobs_application": "Web/CloudOps",
    "bizitobs_service": "Web/CloudOps",
    "bizitobs_l2c/5T": "Web/CloudOps",
    "bizitobs_l2c/H": "Web/CloudOps",
    
    # Sales
    "restaurant": "Sales",
    "car_parts_with_missing": "Sales",
    "kdd_cup_2018_with_missing/H": "Sales",
    "kdd_cup_2018_with_missing/D": "Sales",
    "hierarchical_sales/D": "Sales",
    "hierarchical_sales/W": "Sales",
    
    # Healthcare
    "hospital": "Healthcare",
    "covid_deaths": "Healthcare",
    "us_births/D": "Healthcare",
    "us_births/M": "Healthcare",
    "us_births/W": "Healthcare",
    
    # Econ/Fin
    "m4_yearly": "Econ/Fin",
    "m4_quarterly": "Econ/Fin",
    "m4_monthly": "Econ/Fin",
    "m4_weekly": "Econ/Fin",
    "m4_daily": "Econ/Fin",
    "m4_hourly": "Econ/Fin",
    
    # Energy
    "electricity/15T": "Energy",
    "electricity/H": "Energy",
    "electricity/D": "Energy",
    "electricity/W": "Energy",
    "solar/10T": "Energy",
    "solar/H": "Energy",
    "solar/D": "Energy",
    "solar/W": "Energy",
    "ett1/15T": "Energy",
    "ett1/H": "Energy",
    "ett1/D": "Energy",
    "ett1/W": "Energy",
    "ett2/15T": "Energy",
    "ett2/H": "Energy",
    "ett2/D": "Energy",
    "ett2/W": "Energy",
    
    # Transport
    "LOOP_SEATTLE/5T": "Transport",
    "LOOP_SEATTLE/H": "Transport",
    "LOOP_SEATTLE/D": "Transport",
    "SZ_TAXI/15T": "Transport",
    "SZ_TAXI/H": "Transport",
    "M_DENSE/H": "Transport",
    "M_DENSE/D": "Transport"
}


# Utils

In [8]:
def load_results(base_dir, ):
    # Initialize an empty DataFrame
    all_data = []

    # Walk through subfolders and find horizons_results.csv
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file == 'horizons_results.csv':
                file_path = os.path.join(root, file)
                # Read the CSV file
                data = pd.read_csv(file_path)
                
                # Extract the two-level subfolder names
                relative_path = os.path.relpath(root, base_dir)
                subfolders = relative_path.split(os.sep)
                if len(subfolders) >= 2:
                    dataset = f"{subfolders[-2]}/{subfolders[-1]}"
                else:
                    dataset = subfolders[0] if subfolders else "Unknown"
                
                if "ElasTST" in dataset:
                    model_name = "ElasTST"
                elif "PatchTST" in dataset:
                    model_name = "PatchTST"
                
                dataset = dataset.split(f'_{model_name}_')[0]

                # Extract relevant columns and add dataset info
                if 'test_CRPS' in data.columns and 'horizon' in data.columns:
                    temp_data = data[['test_CRPS', 'horizon']].copy()
                    temp_data['dataset'] = dataset
                    temp_data['model'] = model_name
                    
                    if dataset in dataset_domain_mapping:
                        temp_data['domain'] = dataset_domain_mapping[dataset]
                    else:
                        temp_data['domain'] = 'None'
                    all_data.append(temp_data)


    # Combine all data into a single DataFrame
    result_df = pd.concat(all_data, ignore_index=True)
    
    # 使用 pivot_table 重塑表格
    result_df = result_df.pivot_table(
        index=["dataset", "horizon",'domain'],
        columns="model",
        values="test_CRPS",
        aggfunc="mean"
    ).reset_index()
    
    result_df.columns.name = None 
    
    return result_df

In [ ]:
base_dir = '/data/Blob_WestJP/v-jiawezhang/log/gift_eval/'

load_results(base_dir)

,dataset,horizon,domain,ElasTST,PatchTST
0,LOOP_SEATTLE/5T,48,Transport,0.050091,0.051197
1,LOOP_SEATTLE/D,30,Transport,0.027683,0.030313
2,LOOP_SEATTLE/H,48,Transport,0.063004,0.082483
3,M_DENSE/D,30,Transport,0.064277,0.106457
4,M_DENSE/H,48,Transport,0.173205,0.242996
5,SZ_TAXI/15T,48,Transport,0.091796,0.099854
6,SZ_TAXI/H,48,Transport,0.062541,0.074239
7,bitbrains_fast_storage/5T,48,Web/CloudOps,0.029249,0.035770
8,bitbrains_fast_storage/H,48,Web/CloudOps,0.036748,0.025118
9,bitbrains_rnd/5T,48,Web/CloudOps,0.059544,0.067555
